<a href="https://colab.research.google.com/github/vivi928/m.m/blob/main/%E6%89%BE%E5%9C%96%E7%89%87_%E4%B8%A6%E4%B8%8B%E8%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import os
from webdriver_manager.chrome import ChromeDriverManager

# 設定搜尋關鍵字和下載資料夾
search_query = "green outfit for female office worker 25-35"
output_directory = "green_images"

# 建立存放圖片的資料夾
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 初始化 Selenium WebDriver
def initialize_browser():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# 搜尋圖片並下載
def download_images():
    driver = initialize_browser()
    driver.get("https://images.google.com/")

    # 搜尋圖片
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # 增加等待時間，確保搜尋結果加載

    # 滾動頁面以加載更多圖片
    for _ in range(10):  # 增加滾動的次數
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        time.sleep(2)

    # 抓取圖片元素
    images = driver.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")  # Google 圖片的 CSS 選擇器
    count = 0
    print(f"找到 {len(images)} 張圖片")  # 打印出找到的圖片數量

    for img in images:
        try:
            # 嘗試從 img 元素中抓取高解析度圖片的 URL
            img_url = img.get_attribute("data-src") or img.get_attribute("src") or img.get_attribute("srcset")
            print(f"圖片 URL: {img_url}")  # 打印出圖片 URL 以供檢查

            if img_url and "http" in img_url:
                # 下載圖片
                response = requests.get(img_url, stream=True)
                if response.status_code == 200:  # 確保請求成功
                    file_name = os.path.join(output_directory, f"image_{count:03d}.jpg")
                    with open(file_name, "wb") as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    print(f"已下載圖片: {file_name}")
                    count += 1
                if count >= 10:  # 限制下載數量
                    break
        except Exception as e:
            print(f"下載圖片失敗: {e}")

    driver.quit()
    print(f"完成下載，共下載 {count} 張圖片到資料夾: {output_directory}")

if __name__ == "__main__":
    download_images()


ModuleNotFoundError: No module named 'webdriver_manager'

In [ ]:
!pip install selenium